In [1]:
import os
import sys

import pandas as pd

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

import datetime
from utils import get_path, DATETIME_FORMAT

In [2]:
from universal_connection import UniversalConnection, DBType

In [3]:
import re

In [4]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [5]:
# set(stopwords.words('russian'))

In [6]:
sql = """
with temp as (
select	a.id app_id,
		a.report_comment,
		case when pc.product_id in (
		            		select id from product
		            		where full_name like '%ВЭБ%') then 'VEBMED'
		        else 'ПРОЧИЕ'
		end promo_type
from appointment a 
left join promotion p
	      on a.promotion_id = p.id
	    left join product_condition pc
	      on pc.id = p.product_condition_id
	      --
	where a.good
      and not (
      	a.patient_id in (
	      	select patient_id
	      	from patient_categories pcat
	      	inner join reference rf
	      	   on rf.id=pcat.reference_id
	      	where rf.code='TEST'
	      	)
	    or lower(a.report_comment)='тест'
      	)
)
select *
from temp 
where promo_type = 'VEBMED'





"""

In [7]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df_sql = connection.query(sql)
connection.close()

2020-06-10 14:33:30 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-10 14:33:30 creating ssh tunnel to 172.16.100.19 as root...
2020-06-10 14:33:30 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 41349
2020-06-10 14:33:30 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-06-10 14:33:30 @telemed query:
                    with temp as (
				    select	a.id app_id,
				    		a.report_comment,
				    		case when pc.product_id in (
				    		            		select id from product
				    		            		where full_name like '%ВЭБ%') then 'VEBMED'
				    		        else 'ПРОЧИЕ'
				    		end promo_type
				    from appointment a 
				    left join promotion p
				    	      on a.promotion_id = p.id
				    	    left join product_condition pc
				    	      on pc.id = p.product_condition_id
				    	 

In [8]:
df_sql.head()

,app_id,report_comment,promo_type
0,127919,О.ринит? у ребенка 3-х мес жизни\nРекомендован...,VEBMED
1,144566,"консультация уролога - очная,\r\nузи простаты,...",VEBMED
2,135952,Диагностическая гипотеза : НМЦ по типу олигоме...,VEBMED
3,253668,жалобы на повышение АД до 160/90\nРекомендован...,VEBMED
4,254688,диагностическая гипотеза : остеохондроз грудно...,VEBMED


In [9]:
d = dict()

In [10]:
stop_words = set(stopwords.words('russian'))

In [11]:
for i in range(df_sql.shape[0]):
    temp = df_sql['report_comment'][i]
    temp = re.sub(r'[^\w\s]+|[\d]+', r'',temp.lower()).replace('ё', 'е').strip().split()
    temp = [x for x in temp if not x in stop_words]
    temp = [x for x in temp if len(x) > 2]
    
    for word in temp:
        d[word] = d.get(word, 0) + 1

In [12]:
words = pd.DataFrame.from_dict(d, orient='index').sort_index().reset_index().rename(columns={'index':'word', 0:'amount'})

In [13]:
words = words.sort_values(by=['amount'], ascending=False).reset_index(drop=True)

In [38]:
words

,word,amount
0,крови,19774
1,анализ,18781
2,узи,15410
3,очная,15064
4,осмотр,14568
...,...,...
120196,массивными,1
120197,массивным,1
120198,массивные,1
120199,массивное,1


In [37]:
words.to_excel('words_new.xlsx', index=False)

In [74]:
#df_sql.to_excel('full-comments.xlsx', index=False)

In [75]:
#df_sql[df_sql.report_comment.str.contains('озвоночни') == True].to_excel('Позвоночник.xlsx', index=False)

In [27]:
int_words = []

In [28]:
a = open('interesting_words.txt')
for line in a:
    #print(line)
    int_words.append(line.strip())
a.close()

In [29]:
int_words

['грудной',
 'пневмония',
 'кашель',
 'дыхания',
 'дыхательных',
 'отхаркивающие',
 'одышка',
 'мокроты',
 'дыхательная']

In [30]:
d_int_words = dict()

In [31]:
for i in range(df_sql.shape[0]):
    if any(word in df_sql['report_comment'][i] for word in int_words):
        
        temp = df_sql['report_comment'][i]
        temp = re.sub(r'[^\w\s]+|[\d]+', r'',temp.lower()).replace('ё', 'е').strip().split()
        temp = [x for x in temp if not x in stop_words]
        temp = [x for x in temp if len(x) > 2]

        for word in temp:
            d_int_words[word] = d_int_words.get(word, 0) + 1

In [ ]:
# if any(number < 10 for number in numbers):

In [32]:
df_int_words = pd.DataFrame.from_dict(d_int_words, orient='index').sort_index().reset_index().rename(columns={'index':'word', 0:'amount'})

In [33]:
df_int_words = df_int_words.sort_values(by=['amount'], ascending=False).reset_index(drop=True)

In [34]:
df_int_words

,word,amount
0,температуры,2049
1,анализ,1939
2,грудной,1911
3,очный,1845
4,крови,1799
...,...,...
23054,мельников,1
23055,мельникова,1
23056,мембран,1
23057,мемлсо,1


In [39]:
df_int_words.to_excel('df_int_words.xlsx', index=False)